In [6]:
import cv2
import numpy as np

In [7]:
def getHomography(in_A, in_B, out_A, out_B, threshold, K):
    # 2.1 ，根据inlier获取新的homography matrix和mask
    # 并生成新的inlier集合
    H, mask = cv2.findHomography(in_A, in_B, cv2.RANSAC, threshold)
    new_inA = in_A[mask]
    new_inB = in_B[mask]
    
    # 2.2，循环计算outlier中点间的距离，小于设置的threshold值的对应关键点加入inlier，否则继续为outlier
    for i in len(out_A):
        for j in len(out_B):
            distance = np.linalg.norm(np.array(out_B[j]) - cv2.convertPointsFromHomogeneous(H * np.array(out_A[i])))
            if distance < threshold:
                new_inA.append(out_A[i])
                new_inB.append(out_B[j])
                del out_A[i],out_B[j]
                continue
    
    # 2.3，设置终止迭代条件：当inlier点个数超过outlier时或者迭代次数大于100时结束迭代
    K += 1
    if len(new_inA) / len(out_A) > 1 or K > 100:
        return H
    else:
        return getHomography(new_inA, new_inB, out_A, out_B, threshold)


In [8]:
def ransacMatching(A, B):
    # 1，取四对关键点，设置阈值为5
    inliers_A = A[:4]
    inliers_B = B[:4]
    outlier_A = A[4:]
    outlier_B = B[4:]
    threshold = 5
    K = 0
    # 2， 调用迭代方法获取homography matrix
    findHomography(inliers_A, inliers_B, outlier_A, outlier_B, threshold，K)
        